<a href="https://colab.research.google.com/github/indiegolab/nlp_class/blob/master/rnn_hello(191203).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#tensorflow 2.0 임포트
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

TensorFlow 2.x selected.


In [0]:
tf.__version__

'2.0.0'

In [0]:
import numpy as np
import tensorflow as tf

idx2char = ['토', '마', '를', '먹', '자']

# # Teach hello: hihell -> ihello
# # x_data = [[0, 1, 0, 2, 3, 3]]  # hihell
# y_data = [[1, 0, 2, 3, 3, 4]]  # ihello

# 토마토 가르치기: 토토마를자먹 -> 토마토를먹자
# x_data = [[0, 0, 1, 2, 4, 3]]   ## 토토마를자먹
y_data = [[0, 1, 0, 2, 3, 4]]    # 토마토를먹자

num_classes = 5
input_dim = 5  # one-hot size, same as hidden_size to directly predict one-hot
sequence_length = 6  # |토마토를먹자| == 6
learning_rate = 0.1

x_one_hot = np.array([[[1, 0, 0, 0, 0],   # 토 0
                       [1, 0, 0, 0, 0],   # 토 0
                       [0, 1, 0, 0, 0],   # 마 1
                       [0, 0, 1, 0, 0],   # 를 2
                       [0, 0, 0, 0, 1],   # 자 4
                       [0, 0, 0, 1, 0]]], # 먹 3
                     dtype=np.float32)

y_one_hot = tf.keras.utils.to_categorical(y_data, num_classes=num_classes)
print("x_one_hot의 shape", x_one_hot.shape)
print("y_one_hot의 shape", y_one_hot.shape)

x_one_hot의 shape (1, 6, 5)
y_one_hot의 shape (1, 6, 5)


In [0]:
tf.model = tf.keras.Sequential()
'''
셀을 만들고 RNN 레이어에 추가
input_shape = (1,6,5) => 시퀀스 수 (배치), 시퀀스 길이, dim 사이즈
'''
# cell = tf.keras.layers.LSTMCell(units=num_classes, input_shape=(sequence_length, input_dim))

cell = tf.keras.layers.SimpleRNNCell(units=num_classes, input_shape=(sequence_length, input_dim))

# cell = tf.keras.layers.GRUCell(units=num_classes, input_shape=(sequence_length, input_dim))

# print(num_classes, sequence_length, input_dim)
## 레이어 생성 자체를 GRU나 LSTM로 생성할 수 도 있고, Cell만 따로 만들어서 RNN 레이어에 넣을 수도 있다.

tf.model.add(tf.keras.layers.RNN(cell=cell, # cell 종류만 바꿔서 실험 가능.
                                 return_sequences=True))
# LSTMCell을 만드는 대신 단일 LSTM 레이어를 사용할 수도 있다.
# tf.model.add(tf.keras.layers.LSTM(units=num_classes, input_shape=(sequence_length, input_dim), return_sequences=True))

In [14]:
# Fully Connected layer, 활성함수 = softmax
tf.model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units=num_classes, activation='softmax')))

# 손실함수 = 'categorical_crossentropy, Adam 옵티마이저 사용
tf.model.compile(loss='categorical_crossentropy', 
                 optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
                 metrics=['accuracy'])

# 훈련
tf.model.fit(x_one_hot, y_one_hot, epochs=50) # 에폭 = 50
tf.model.summary()

predictions = tf.model.predict(x_one_hot)
for i, prediction in enumerate(predictions):
    print(prediction)

    # print char using argmax, dict
    result_str = [idx2char[c] for c in np.argmax(prediction, axis=1)]
    print("\tPrediction str: ", ''.join(result_str))

Train on 1 samples
Epoch 1/50
1/1 [==============================] - 1s 1s/sample - loss: 1.9842 - accuracy: 0.0000e+00
Epoch 2/50
1/1 [==============================] - 0s 19ms/sample - loss: 1.7828 - accuracy: 0.0000e+00
Epoch 3/50
1/1 [==============================] - 0s 13ms/sample - loss: 1.5969 - accuracy: 0.0000e+00
Epoch 4/50
1/1 [==============================] - 0s 14ms/sample - loss: 1.4151 - accuracy: 0.5000
Epoch 5/50
1/1 [==============================] - 0s 14ms/sample - loss: 1.2785 - accuracy: 0.6667
Epoch 6/50
1/1 [==============================] - 0s 13ms/sample - loss: 1.1617 - accuracy: 0.6667
Epoch 7/50
1/1 [==============================] - 0s 14ms/sample - loss: 1.0536 - accuracy: 0.6667
Epoch 8/50
1/1 [==============================] - 0s 12ms/sample - loss: 0.9566 - accuracy: 0.8333
Epoch 9/50
1/1 [==============================] - 0s 13ms/sample - loss: 0.8688 - accuracy: 0.8333
Epoch 10/50
1/1 [==============================] - 0s 14ms/sample - loss: 0.7869